# Tutorial: Zero-Shot Text Classification

In this short tutorial, we show how to use *ferret* to use and evaluate different explainability approaches in the task of Zero-Shot Text Classification.

We will use `MoritzLaurer/mDeBERTa-v3-base-mnli-xnli` as model checkpoint.

In [ ]:
%%capture
!pip install -U ferret-xai

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from ferret import (
    Benchmark,
    GradientExplainer,
    IntegratedGradientExplainer,
    LIMEExplainer,
    SHAPExplainer,
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

In [ ]:
ig = IntegratedGradientExplainer(model, tokenizer, multiply_by_inputs=True)
g = GradientExplainer(model, tokenizer, multiply_by_inputs=True)
l = LIMEExplainer(model, tokenizer)

In [ ]:
bench = Benchmark(
    model, tokenizer, task_name="zero-shot-text-classification", explainers=[ig, g, l]
)

In [ ]:
sequence_to_classify = (
    "Amanda ha cucinato la più buona torta pecan che abbia mai provato!"
)
candidate_labels = ["politics", "economy", "bakery"]
sample = (sequence_to_classify, candidate_labels)

In [ ]:
# get the prediction from our model
bench.score(sample, return_probs=True)

In [ ]:
# explain the contradiction class
exp = bench.explain(sample, target="entailment")

In [ ]:
# show explanations
bench.show_table(exp)

In [ ]:
# evaluate explanations and show faithfulness metrics
bench.show_evaluation_table(bench.evaluate_explanations(exp, target="contradiction"))